The Link for dataset is https://www.kaggle.com/datasets/grouplens/movielens-20m-dataset?select=link.csv

In [ ]:
from google.colab import drive
import sys

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df_movie=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/teaching/gwu/machine_learning_I/project/movie_recommender /Data/Movie Lens/movie.csv')

df_ratings=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/teaching/gwu/machine_learning_I/project/movie_recommender /Data/Movie Lens/rating.csv')

df_link=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/teaching/gwu/machine_learning_I/project/movie_recommender /Data/Movie Lens/link.csv')

In [ ]:
df_link=df_link.drop('movieId',axis=1)

In [ ]:
df_movies=pd.concat([df_movie,df_link],axis=1)

In [ ]:
df_movies.head()

,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0


In [ ]:
import re

def clean_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title

In [ ]:
df_movies["clean_title"] = df_movies["title"].apply(clean_title)


In [ ]:
df_movies.head()

,movieId,title,genres,imdbId,tmdbId,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0,Father of the Bride Part II 1995


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer =TfidfVectorizer(ngram_range=(1,2))

tfidf=vectorizer.fit_transform(df_movies['clean_title'].values.astype('U'))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = df_movies.iloc[indices].iloc[::-1]
    
    return results

In [ ]:
import ipywidgets as widgets
from IPython.display import display

movie_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            display(search(title))

movie_input.observe(on_type, names='value')


display(movie_input, movie_list)

Text(value='Toy Story', description='Movie Title:')

Output()

In [ ]:
movie_id=1

In [ ]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [ ]:
similar_users = df_ratings[(df_ratings["movieId"] == movie_id) & (df_ratings["rating"] > 4)]["userId"].unique()

In [ ]:
similar_users

array([     6,     11,     14, ..., 138473, 138474, 138486])

In [ ]:
similar_user_recs = df_ratings[(df_ratings["userId"].isin(similar_users)) & (df_ratings["rating"] > 4)]["movieId"]

In [ ]:
similar_user_recs

517             1
519             7
520            17
521            52
522            62
            ...  
19999505    53000
19999506    53953
19999507    54771
19999509    55282
19999510    56174
Name: movieId, Length: 1038237, dtype: int64

In [ ]:
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
similar_user_recs = similar_user_recs[similar_user_recs > .10]

In [ ]:
similar_user_recs

1       1.000000
318     0.426600
260     0.395222
296     0.359822
356     0.340574
          ...   
778     0.101869
733     0.100631
2502    0.100631
2804    0.100384
1207    0.100136
Name: movieId, Length: 104, dtype: float64

In [ ]:
all_users = df_ratings[(df_ratings["movieId"].isin(similar_user_recs.index)) & (df_ratings["rating"] > 4)]

In [ ]:
all_users

,userId,movieId,rating,timestamp
30,1,1196,4.5,2005-04-02 23:32:22
31,1,1198,4.5,2005-04-02 23:30:24
131,1,4993,5.0,2005-04-02 23:31:22
142,1,5952,5.0,2005-04-02 23:30:19
158,1,7153,5.0,2005-04-02 23:30:33
...,...,...,...,...
20000106,138493,4973,5.0,2009-10-17 19:04:58
20000138,138493,6377,5.0,2009-10-28 17:20:00
20000142,138493,6539,4.5,2009-10-17 20:17:37
20000150,138493,6874,5.0,2009-10-17 19:08:23


In [ ]:
all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

In [ ]:
all_user_recs

318     0.318561
296     0.279770
593     0.229719
527     0.220469
356     0.218255
          ...   
1580    0.047004
745     0.044837
551     0.044578
2804    0.042803
2355    0.027781
Name: movieId, Length: 104, dtype: float64

In [ ]:
rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
rec_percentages.columns = ["similar", "all"]

In [ ]:
rec_percentages

,similar,all
1,1.000000,0.126878
32,0.175826,0.111307
34,0.155898,0.065850
47,0.213764,0.139779
50,0.287350,0.209453
...,...,...
7153,0.207204,0.132626
7361,0.121735,0.086273
8961,0.160045,0.054511
33794,0.106016,0.054880


In [ ]:
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]

In [ ]:
rec_percentages = rec_percentages.sort_values("score", ascending=False)

In [ ]:
rec_percentages

,similar,all,score
1,1.000000,0.126878,7.881607
3114,0.269031,0.055484,4.848761
2355,0.116351,0.027781,4.188078
4886,0.189256,0.057903,3.268505
6377,0.189751,0.060918,3.114850
...,...,...,...
296,0.359822,0.279770,1.286134
858,0.256220,0.200352,1.278850
2858,0.222057,0.174455,1.272863
4973,0.127243,0.102221,1.244783


In [ ]:
rec_percentages.head(10).merge(df_movies, left_index=True, right_on="movieId")

,similar,all,score,movieId,title,genres,imdbId,tmdbId,clean_title
0,1.000000,0.126878,7.881607,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,Toy Story 1995
3027,0.269031,0.055484,4.848761,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,120363,863.0,Toy Story 2 1999
2270,0.116351,0.027781,4.188078,2355,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy,120623,9487.0,Bugs Life A 1998
4790,0.189256,0.057903,3.268505,4886,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy,198781,585.0,Monsters Inc 2001
6271,0.189751,0.060918,3.114850,6377,Finding Nemo (2003),Adventure|Animation|Children|Comedy,266543,12.0,Finding Nemo 2003
582,0.212774,0.071731,2.966280,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,103639,812.0,Aladdin 1992
8278,0.160045,0.054511,2.936018,8961,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,317705,9806.0,Incredibles The 2004
589,0.188390,0.067389,2.795573,595,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX,101414,10020.0,Beauty and the Beast 1991
360,0.231279,0.084640,2.732495,364,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,110357,8587.0,Lion King The 1994
1052,0.151133,0.057463,2.630074,1073,Willy Wonka & the Chocolate Factory (1971),Children|Comedy|Fantasy|Musical,67992,252.0,Willy Wonka the Chocolate Factory 1971


In [ ]:
def find_similar_movies(movie_id):
  similar_users = df_ratings[(df_ratings["movieId"] == movie_id) & (df_ratings["rating"] > 4)]["userId"].unique()
  similar_user_recs = df_ratings[(df_ratings["userId"].isin(similar_users)) & (df_ratings["rating"] > 4)]["movieId"]

  similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
  similar_user_recs = similar_user_recs[similar_user_recs > .10]

  all_users = df_ratings[(df_ratings["movieId"].isin(similar_user_recs.index)) & (df_ratings["rating"] > 4)]
  all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

  rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
  rec_percentages.columns = ["similar", "all"]

  rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
  rec_percentages = rec_percentages.sort_values("score", ascending=False)

  return rec_percentages.head(10).merge(df_movies, left_index=True, right_on="movieId")









In [ ]:
movie_name_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()